- Optimized Unary Encoding (OUE) [36] is an optimized Basic One-time RAPPOR method by choosing the optimal parameters and to guarantee low variance.

### Differential Privacy


Read the dataset

In [352]:
import pickle
import pandas as pd


archivo_pickle = 'Entrenamiento_cronicas_N.pkl'

with open(archivo_pickle, 'rb') as file:
    df = pickle.load(file)

Delete some columns 

In [353]:
# Eliminar columnas donde todas las entradas son cero
df = df.loc[:, (df != 0).any(axis=0)]

# Mostrar las columnas eliminadas y el nuevo dataframe
columns_removed = df.columns.difference(df.columns)


In [355]:
# columns_enfermedades = [
#     'SIDA_VIH', 'TUBERCULOSIS', 'CANCER_OTROSCANCER', 'CANCER_ORGANOSDIGESTIVOS',
#     'CANCER_ORGANOSRESPIRATORIOS', 'CANCER_MELANOMAYDELAPIEL', 'CANCER_MAMA',
#     'CANCER_OTROSGENITALESFEMENINOS', 'CANCER_CERVIXINVASIVO', 'CANCER_GENITALESMASCULINOS',
#     'CANCER_TEJIDOLINFATICOYAFINES', 'CANCER_CERVIXINSITU', 'AUTOINMUNE', 'DIABETES',
#     'SINDROMESCONVULSIVOS(EPILEPSIA)', 'ENFERMEDADCARDIOVASCULAR_OTRA', 'ENFERMEDADCARDIOVASCULAR_HIPERTENSION',
#     'ENFERMEDADPULMONARDELARGADURACION', 'ASMA', 'ARTRITISPIOGENASYREACTIVAS',
#     'ARTRITIS', 'ARTROSIS', 'RENAL_OTRA', 'RENAL_RENALDELARGADURACION', 'TRANSPLANTE',
#     'RENAL_INSUFICIENCIARENALOTRA', 'RENAL_INSUFICIENCIARENALCRONICA', 'ANOMALIASGENETICASYCONGENITAS',
#     'CANCER_TERAPIACANCER'
# ]
df.drop(columns='NUM_IDE', inplace=True)
df.drop(columns='FECHA_NACI', inplace=True)
df.drop(columns='id', inplace=True)
df.reset_index(drop=True, inplace=True)

In [356]:
#data_non_zero.drop(columns='Valor_Neto_Chunk', inplace=True)
#data_non_zero.drop(columns='DIAS_ESTAN_Neto_Chunk', inplace=True)
df

,EPS,SEXO,DPTO,MUNI,DIAS_COMP,V8,Valor_Neto,DIAS_ESTAN_Neto,Edad,SIDA_VIH,...,ARTROSIS,RENAL_OTRA,RENAL_RENALDELARGADURACION,TRANSPLANTE,RENAL_INSUFICIENCIARENALOTRA,RENAL_INSUFICIENCIARENALCRONICA,ANOMALIASGENETICASYCONGENITAS,CANCER_TERAPIACANCER,Grupo_Edad,Zona
0,EPS002,F,50,1,360,B,39054.0,0.0,12.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,N
1,EPS013,M,76,1,336,C,312012.0,277.0,23.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,C
2,EPS017,F,11,1,333,B,14313.0,0.0,15.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,C
3,EPS002,M,76,1,84,B,0.0,0.0,13.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,C
4,EPS037,F,15,776,180,B,0.0,0.0,21.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996229,EPS005,M,88,1,360,C,96085.0,35.0,31.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,Z
996230,EPS037,F,76,111,215,B,0.0,0.0,47.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,N
996231,EPS002,M,11,1,30,B,0.0,0.0,30.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,C
996232,EPS013,M,15,1,360,C,70114.0,0.0,32.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,N


In [357]:
possible_categorical_columns = ['EPS', 'SEXO', 'DPTO', 'MUNI', 'V8', 'Grupo_Edad', 'Zona',
    'SIDA_VIH', 'TUBERCULOSIS', 'CANCER_OTROSCANCER', 'CANCER_ORGANOSDIGESTIVOS',
    'CANCER_ORGANOSRESPIRATORIOS', 'CANCER_MELANOMAYDELAPIEL', 'CANCER_MAMA',
    'CANCER_OTROSGENITALESFEMENINOS', 'CANCER_CERVIXINVASIVO', 'CANCER_GENITALESMASCULINOS',
    'CANCER_TEJIDOLINFATICOYAFINES', 'CANCER_CERVIXINSITU', 'AUTOINMUNE', 'DIABETES',
    'SINDROMESCONVULSIVOS(EPILEPSIA)', 'ENFERMEDADCARDIOVASCULAR_OTRA', 'ENFERMEDADCARDIOVASCULAR_HIPERTENSION',
    'ENFERMEDADPULMONARDELARGADURACION', 'ASMA', 'ARTRITISPIOGENASYREACTIVAS',
    'ARTRITIS', 'ARTROSIS', 'RENAL_OTRA', 'RENAL_RENALDELARGADURACION', 'TRANSPLANTE',
    'RENAL_INSUFICIENCIARENALOTRA', 'RENAL_INSUFICIENCIARENALCRONICA', 'ANOMALIASGENETICASYCONGENITAS',
    'CANCER_TERAPIACANCER']

one_hot_encoded=pd.get_dummies(df, columns=possible_categorical_columns)

In [358]:
one_hot_encoded=one_hot_encoded.astype(int)

In [97]:
import torch
import pandas as pd

def laplace_mechanism_torch(values, epsilon):
    """Vectorized Laplace mechanism using PyTorch."""
    scale = 1 / epsilon
    noise = torch.distributions.Laplace(torch.tensor([0.0]), torch.tensor([scale])).sample(values.shape)
    return values + noise.squeeze()

def apply_oue_to_one_hot_torch(data, column, epsilon):
    """Applies Optimized Unary Encoding to a column using PyTorch, converting to one-hot first."""
    categories = pd.get_dummies(data[column], prefix=column)
    category_tensor = torch.tensor(categories.values, dtype=torch.float32)

    p = 0.9  # Probability of retaining the original value
    q = 1 / (torch.exp(torch.tensor(epsilon)) + 1)  # Probability of flipping a 0 to a 1

    # Calculate probabilities for Bernoulli distribution
    pr = category_tensor * p + (1 - category_tensor) * q
    noisy_category = torch.bernoulli(pr)

    # Convert noisy one-hot encoding to unary encoding
    unary_encoding = torch.argmax(noisy_category, dim=1)

    # Convert unary encoding to DataFrame
    new_column_name = f'{column}_encoded'
    return unary_encoding

In [88]:
one_hot_encoded

,DIAS_COMP,Valor_Neto,DIAS_ESTAN_Neto,Edad,EPS_EAS016,EPS_EAS027,EPS_EPS001,EPS_EPS002,EPS_EPS003,EPS_EPS005,...,TRANSPLANTE_0.0,TRANSPLANTE_1.0,RENAL_INSUFICIENCIARENALOTRA_0.0,RENAL_INSUFICIENCIARENALOTRA_1.0,RENAL_INSUFICIENCIARENALCRONICA_0.0,RENAL_INSUFICIENCIARENALCRONICA_1.0,ANOMALIASGENETICASYCONGENITAS_0.0,ANOMALIASGENETICASYCONGENITAS_1.0,CANCER_TERAPIACANCER_0.0,CANCER_TERAPIACANCER_1.0
0,360,39054,0,12,0,0,0,1,0,0,...,1,0,1,0,1,0,1,0,1,0
1,336,312012,277,23,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,1,0
2,333,14313,0,15,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,1,0
3,84,0,0,13,0,0,0,1,0,0,...,1,0,1,0,1,0,1,0,1,0
4,180,0,0,21,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996229,360,96085,35,31,0,0,0,0,0,1,...,1,0,1,0,1,0,1,0,1,0
996230,215,0,0,47,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,1,0
996231,30,0,0,30,0,0,0,1,0,0,...,1,0,1,0,1,0,1,0,1,0
996232,360,70114,0,32,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,1,0


In [103]:
import torch
import pandas as pd

# Sample DataFrame with one-hot encoded data
# Assuming you have a column with only values 0 or 1



def hamming_distance(vector1, vector2):
    # Compute bitwise XOR to find differing bits
    xor_result = vector1 ^ vector2
    
    # Count the number of differing bits
    num_diff_bits = torch.sum(xor_result)
    
    return num_diff_bits.item()


epsilon = 13232


res_eps_ea016 = apply_oue_to_one_hot_torch(one_hot_encoded,'EPS_EAS016', epsilon)

eps_ea016=torch.tensor(one_hot_encoded['EPS_EAS016'])
#calculate the hamming distance between the original and the noisy data
hamming_distance(eps_ea016,res_eps_ea016)


47

In [29]:
for col in categorical_columns:
    unique_values = df[col].nunique()
    print(f"{col} has {unique_values} unique categories.")


ANOMALIASGENETICASYCONGENITAS_0.0 has 2 unique categories.
ANOMALIASGENETICASYCONGENITAS_1.0 has 2 unique categories.
ARTRITISPIOGENASYREACTIVAS_0.0 has 2 unique categories.
ARTRITISPIOGENASYREACTIVAS_1.0 has 2 unique categories.
ARTRITIS_0.0 has 2 unique categories.
ARTRITIS_1.0 has 2 unique categories.
ARTROSIS_0.0 has 2 unique categories.
ARTROSIS_1.0 has 2 unique categories.
ASMA_0.0 has 2 unique categories.
ASMA_1.0 has 2 unique categories.
AUTOINMUNE_0.0 has 2 unique categories.
AUTOINMUNE_1.0 has 2 unique categories.
CANCER_CERVIXINSITU_0.0 has 2 unique categories.
CANCER_CERVIXINSITU_1.0 has 2 unique categories.
CANCER_CERVIXINVASIVO_0.0 has 2 unique categories.
CANCER_CERVIXINVASIVO_1.0 has 2 unique categories.
CANCER_GENITALESMASCULINOS_0.0 has 2 unique categories.
CANCER_GENITALESMASCULINOS_1.0 has 2 unique categories.
CANCER_MAMA_0.0 has 2 unique categories.
CANCER_MAMA_1.0 has 2 unique categories.
CANCER_MELANOMAYDELAPIEL_0.0 has 2 unique categories.
CANCER_MELANOMAYDELAPI

### Randomized response with coin flips

es un algoritmo aleatorio. Una forma de describir la privacidad de un algoritmo es hasta qué punto la contribución de un individuo puede afectar los resultados finales. Más formalmente, sea MM un algoritmo aleatorio. MM es ϵϵ-diferencialmente privado si para cualquier ss en el rango de $M$ y para cualquier par de bases de datos $d,d′$ en el dominio de $M$ que difieren en solo una fila, $P(M(d)=s)/P(M(d′)=s)≤eϵP(M(d)=s)/P(M(d′)=s)≤e^{\epsilon}$. Cuanto menos probable sea obtener el mismo resultado con una base de datos ligeramente modificada, mayor necesitaríamos hacer ϵϵ.

In [359]:
def count_different_categories(original_data, randomized_responses):
    """
    Counts the number of categories in which the original data and randomized responses are different.
    
    Parameters:
        original_data (pd.Series): Pandas Series containing original categorical data.
        randomized_responses (pd.Series): Pandas Series containing randomized responses.
    
    Returns:
        int: Number of categories in which the original data and randomized responses are different.
    """
    # Count the number of differences between the original data and randomized responses
    num_differences = (original_data != randomized_responses).sum()
    return num_differences

In [360]:
def apply_randomized_response_torch(data, categories, epsilon):
    num_categories = len(categories)
    e_eps = np.exp(epsilon)
    p = e_eps / (e_eps + num_categories - 1)
    
    print(f"Number of categories (k): {num_categories}")
    print(f"Probability of truthful response (p): {p:.4f}")

    data_indices = torch.tensor([categories.tolist().index(cat) for cat in data], dtype=torch.long)
    random_indices = torch.randint(0, num_categories, (len(data),))
    random_probs = torch.rand(len(data))
    final_indices = torch.where(random_probs < p, data_indices, random_indices)
    randomized_responses = np.array(categories)[final_indices.numpy()]

    return pd.Series(randomized_responses, index=data.index)



In [366]:
data=df['EPS']
# Example usage
epsilon = np.log(23)  # Privacy budget


randomized_responses = apply_randomized_response_torch(data, data.unique(), epsilon)
randomized_responses
print(data)
print(count_different_categories(data, randomized_responses))



Number of categories (k): 24
Probability of truthful response (p): 0.5000
0         EPS002
1         EPS013
2         EPS017
3         EPS002
4         EPS037
           ...  
996229    EPS005
996230    EPS037
996231    EPS002
996232    EPS013
996233    EPS037
Name: EPS, Length: 996234, dtype: object
477281


In [367]:
len(data.unique())

24

Differential Privacy in Randomized Response

In differential privacy, ϵϵ controls the privacy guarantee of the mechanism, dictating the likelihood ratio between the probabilities of outputs given two adjacent inputs (differing in only one entry):

    A lower ϵϵ provides stronger privacy by making the outputs more uniform (less dependent on the input).
    A higher ϵϵ tends to retain more information about the input, providing less privacy.

Calculating ϵϵ for a Multi-Category Randomized Response

The correct approach to setting ϵϵ in a multi-category randomized response is to ensure that the mechanism’s outputs do not allow one to infer too much about the original input. Here's the correct calculation:

    Probability of truthful response (pp) and probability of a random response (qq) are given by:
    p=eϵeϵ+k−1
    p=eϵ+k−1eϵ​
    q=1eϵ+k−1
    q=eϵ+k−11​

    where kk is the number of categories.

    Solving for ϵϵ when p=0.5p=0.5 in a setup with kk categories:
    p=eϵeϵ+k−1=0.5
    p=eϵ+k−1eϵ​=0.5
    eϵ=k−1
    eϵ=k−1
    ϵ=ln⁡(k−1)
    ϵ=ln(k−1)

For 24 categories, this would be:
ϵ=ln⁡(24−1)=ln⁡(23)
ϵ=ln(24−1)=ln(23)

In [368]:
import numpy as np
from scipy.optimize import minimize
from scipy.special import softmax  # Softmax ensures that estimated proportions sum to 1

def log_likelihood(proportions, observations, p, k):
    """ Log-Likelihood function for randomized response with multiple categories. """
    # Ensure proportions sum to 1 and are non-negative
    proportions = softmax(proportions)  # Softmax to maintain proportion constraints
    expected_probs = p * proportions + (1 - p) / k
    # Log likelihood calculation
    return -np.sum(np.log(expected_probs[observations]))

def estimate_proportions(observations, p, k):
    """ Estimator for true proportions under randomized response. """
    initial_guess = np.full(k, 1/k)  # Uniform initial guess
    result = minimize(
        log_likelihood, 
        initial_guess, 
        args=(observations, p, k),
        method='L-BFGS-B',
        bounds=[(0, 1)] * k
    )
    return softmax(result.x)  # Return softmax to ensure they sum to 1

# Example usage

k=len(data.unique())  # Number of unique categories in the 'EPS' column
# Simulated observed data
p = 0.7  # Probability of answering truthfully

estimated_props = estimate_proportions(data, p, k)
print(f"Estimated category proportions: {estimated_props}")



IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [385]:
import numpy as np

def rappor_encode(value, categories, num_hashes, num_bits):
    """Encode the value using a simple Bloom filter."""
    bit_array = np.zeros(num_bits)
    base_hash = hash(value)
    for i in range(num_hashes):
        hash_val = (base_hash + i) % num_bits
        bit_array[hash_val] = 1
    return bit_array

def rappor_permanent_randomized(bit_array, f):
    """Apply permanent randomized response to the bit array."""
    perturbed = np.random.binomial(1, f, size=bit_array.size)
    return np.logical_xor(bit_array, perturbed).astype(int)

def rappor_instantaneous_randomized(permanent, p):
    """Apply instantaneous randomized response to the permanent bit array."""
    random_flip = np.random.binomial(1, p, size=permanent.size)
    return np.logical_xor(permanent, random_flip).astype(int)

def simulate_rappor(data, categories, f, p, num_hashes, num_bits):
    """Simulate the RAPPOR mechanism across a dataset."""
    encoded_data = [rappor_encode(val, categories, num_hashes, num_bits) for val in data]
    permanent_responses = [rappor_permanent_randomized(arr, f) for arr in encoded_data]
    instantaneous_responses = [rappor_instantaneous_randomized(arr, p) for arr in permanent_responses]
    return instantaneous_responses

# Example usage
categories = ['Apple', 'Banana', 'Cherry']
data = ['Apple'] * 50 + ['Banana'] * 30 + ['Cherry'] * 20
num_bits = 10  # Number of bits in the Bloom filter
num_hashes = 3  # Number of hash functions
f = 0.25  # Probability of flipping in permanent response
p = 0.10  # Probability of flipping in instantaneous response

responses = simulate_rappor(data, categories, f, p, num_hashes, num_bits)
print("Sample of RAPPOR responses:", responses[:5])


Sample of RAPPOR responses: [array([0, 0, 1, 1, 1, 1, 0, 0, 0, 0]), array([1, 1, 0, 1, 0, 1, 1, 0, 0, 0]), array([0, 0, 0, 1, 1, 1, 1, 1, 0, 0]), array([0, 0, 1, 1, 0, 1, 0, 1, 0, 0]), array([1, 0, 0, 0, 1, 0, 0, 1, 0, 1])]


In [386]:
pip install tensorflow-privacy

     ---------------------------------------- 0.0/46.2 kB ? eta -:--:--
     ---------------------------------------- 46.2/46.2 kB 2.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/323.2 kB ? eta -:--:--
   --------------------------- ------------ 225.3/323.2 kB 6.9 MB/s eta 0:00:01
   ---------------------------------------- 323.2/323.2 kB 5.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/126.5 kB ? eta -:--:--
   ---------------------------------------- 126.5/126.5 kB 3.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/101.3 kB ? eta -:--:--
   ---------------------------------------- 101.3/101.3 kB 2.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/104.8 kB ? eta -:--:--
   ---------------------------------------- 104.8/104.8 kB 5.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
    --------------------------------------- 0.2/10.6 MB 5.3 MB/s eta 0:00:02
   - ------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-cpu 2.10.0 requires tensorflow-intel==2.10.0; platform_system == "Windows", but you have tensorflow-intel 2.16.1 which is incompatible.


In [387]:
input_layer = tf.reshape(features['x'], [-1, 28, 28, 1])
y = tf.keras.layers.Conv2D(16, 8,
                           strides=2,
                           padding='same',
                           activation='relu').apply(input_layer)
y = tf.keras.layers.MaxPool2D(2, 1).apply(y)
y = tf.keras.layers.Conv2D(32, 4,
                           strides=2,
                           padding='valid',
                           activation='relu').apply(y)
y = tf.keras.layers.MaxPool2D(2, 1).apply(y)
y = tf.keras.layers.Flatten().apply(y)
y = tf.keras.layers.Dense(32, activation='relu').apply(y)
logits = tf.keras.layers.Dense(10).apply(y)
predicted_labels = tf.argmax(input=logits, axis=1)

NameError: name 'tf' is not defined

In [423]:
import numpy as np

def duchi_algorithm(li, epsilon):
    d = len(li)  # Dimension of the input tuple
    t = np.zeros(d)  # Initialize random tuple t
    
    # Step 1: Generate the random tuple t
    for i in range(d):
        sign = 1 if li[i] >= 0 else -1
        t[i] = sign * (1/2 + abs(li[i]))
    
    # Step 2: Calculate the dot product and classify
    u = np.random.normal(0, 1, d)  # You can use other distributions for u as appropriate
    dot_product = np.dot(t, u)
    
    T_plus = t if dot_product >= 0 else None
    T_minus = t if dot_product < 0 else None
    
    # Step 3: Sample a Bernoulli variable u
    p = np.exp(epsilon) / (np.exp(epsilon) + 1)
    bernoulli_sample = np.random.binomial(1, p)
    
    # Step 4: Return a tuple based on the Bernoulli outcome
    if bernoulli_sample == 1 and T_plus is not None:
        return T_plus
    elif bernoulli_sample == 0 and T_minus is not None:
        return T_minus
    else:
        return None  # Handling the case where the respective set is None

# Example usage
li=data['valor_neto'] # Example multidimensional data
epsilon = 30  # Privacy parameter
output = duchi_algorithm(li, epsilon)
print("Output tuple:", output)


Output tuple: [-1.5  1.5  1.  -1. ]


In [17]:
import numpy as np

def one_hot_encode(categories, values):
    """One-hot encode the categorical values."""
    category_dict = {category: idx for idx, category in enumerate(categories)}
    encoded = np.zeros((len(values), len(categories)))
    for i, value in enumerate(values):
        encoded[i, category_dict[value]] = 1
    return encoded, category_dict

def decode_one_hot(encoded_values, category_dict):
    """Decode one-hot encoded values to categorical values."""
    reverse_category_dict = {idx: category for category, idx in category_dict.items()}
    decoded = [reverse_category_dict[np.argmax(vector)] for vector in encoded_values]
    return decoded

def perturb_bit(bit, p, q):
    """Perturb a single bit using probabilities p and q."""
    if bit == 1:
        return np.random.choice([1, 0], p=[p, 1 - p])
    else:
        return np.random.choice([1, 0], p=[q, 1 - q])

def oue_perturb(vector, epsilon):
    """Apply OUE to perturb a binary vector."""
    p = np.exp(epsilon) / (1 + np.exp(epsilon))
    q = 1 / (1 + np.exp(epsilon))
    perturbed_vector = [perturb_bit(bit, p, q) for bit in vector]
    return perturbed_vector

# Example categorical data
categories = ["Red", "Green", "Blue"]
values = ["Red", "Green", "Blue", "Green", "Red"]

# One-hot encode the categorical values
encoded_values, category_dict = one_hot_encode(categories, values)

# Apply OUE to each one-hot encoded vector
epsilon = 1.0  # Example privacy budget
perturbed_values = [oue_perturb(vector, epsilon) for vector in encoded_values]

# Decode the perturbed values back to categories
decoded_perturbed_values = decode_one_hot(perturbed_values, category_dict)

# Print the original, perturbed, and decoded values
print("Original One-Hot Encoded Values:")
print(encoded_values)
print("Perturbed One-Hot Encoded Values:")
print(perturbed_values)
print("Decoded Perturbed Categories:")
print(decoded_perturbed_values)



Original One-Hot Encoded Values:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]
Perturbed One-Hot Encoded Values:
[[0, 0, 0], [0, 1, 1], [0, 0, 1], [0, 0, 0], [1, 1, 0]]
Decoded Perturbed Categories:
['Red', 'Green', 'Blue', 'Red', 'Red']
